In [ ]:
using DelaySSAToolkit, Catalyst
using Test,BenchmarkTools

In [ ]:
rn = @reaction_network begin
    1/(1+Y^2), 0 --> X
    1/(1+Y),   Y --> 0
end
jumpsys = convert(JumpSystem, rn, combinatoric_ratelaws = false)

In [ ]:
u0 = [0,0]
de_chan0 = [[]]
tf = 400.
tspan = (0,tf)
τ = 20.
dprob = DiscreteProblem(jumpsys, u0, tspan)

In [ ]:
delay_trigger_affect! = function (integrator, rng)
    append!(integrator.de_chan[1], τ)
  end
  delay_trigger = Dict(1=>delay_trigger_affect!)
  delay_complete = Dict(1=>[2=>1, 1=>-1], 2=>[2=>1])
  delay_interrupt = Dict()
  delayjumpset = DelayJumpSet(delay_trigger, delay_complete, delay_interrupt)

In [10]:
medianlist=[]
algo_list =[DelayDirect(),DelayRejection(),DelayMNRM(),DelayDirectCR()]
for algo in algo_list
    djprob = DelayJumpProblem(jumpsys, dprob,algo,delayjumpset, de_chan0, save_positions=(false,false))
    a=@benchmark solve(djprob, SSAStepper())
    push!(medianlist,median(a).time/1e9)
end
medianlist

In [ ]:
p1=bar(algo_name,medianlist,legend=:false,title="oscillation",ylabel="mediantime")
scatter!(algo_name, 0.0003 .+ medianlist , markeralpha=0, series_annotations=medianvalue)

In [ ]:
using Plots

In [ ]:
medianvalue=[string(round(mt,digits=6),"s") for mt in medianlist]
algo_name = ["DelayDirect","DelayRejection","DelayMNRM","DelayDirectCR"]